### viewers setup

In [40]:
import os
from os.path import join as joinp
from brainbuilder.utils import viewer
from IPython.display import HTML,display
from IPython.html import widgets
#location where output will be stored under viewer directory:
directory = "out"

# the output directory should be in the viewer directory to be accessible by the viewer.
output_directory = joinp("..", "viewer", directory)

if not os.path.isdir(output_directory):
    os.makedirs(output_directory)
    
#change display_server to the one that serves the viewer. 
display_server = "http://bbp.courcol.ch:8082/"
display_path = display_server + '#' + directory + '/'

iframe = '<iframe src="'+ display_path + '%s"  scrolling="no" width=700 height=350></iframe>'

### loaders initialization

In [41]:
from brainbuilder.utils import genbrain as gb
from brainbuilder.utils import bbp
from brainbuilder.utils import traits as tt

# Brain Builder modules

##   Build.Region: Region of Interest

In [42]:
from brainbuilder.select_region import select_region

### input parameters

In [43]:
annotation = gb.MetaIO.load("../data/P56_Mouse_annotation/annotation.mhd")
hierarchy = gb.load_hierarchy("../data/P56_Mouse_annotation/annotation_hierarchy.json")['msg'][0]
full_density = gb.MetaIO.load("../data/atlasVolume/atlasVolume.mhd")

d = widgets.Dropdown(options= ["Primary somatosensory area, lower limb",
                    "Primary somatosensory area",
                    "Somatosensory areas"])
display(d)

In [44]:
region_name = d.value

### run module

In [45]:
density_raw = select_region(annotation.raw, full_density.raw, hierarchy, region_name)

### output

In [48]:
import copy
selected_region_mhd = copy.copy(full_density.mhd);
selected_region_mhd['ElementDataFile'] = 'selected_region.raw'
gb.save_meta_io(joinp(output_directory,'selected_region.mhd'),selected_region_mhd, joinp(output_directory,'selected_region.raw'), density_raw)
HTML(iframe % "selected_region.mhd")

##  Build.Cells:  Cell Positions

In [28]:
from brainbuilder.cell_positioning import cell_positioning

### input parameters

In [29]:
voxel_dimensions = full_density.mhd['ElementSpacing']
total_cell_count = 1000

### run module

In [30]:
positions = cell_positioning(density_raw, voxel_dimensions, total_cell_count)

### output

In [38]:
viewer.serialize_points(joinp(output_directory,'positions.pts'), positions, [[50, 50, 50]]*len(positions))
HTML(iframe % "positions.pts")

## Build.EI:  E-I ratios

In [73]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist

### input parameters

In [74]:
recipe_filename = "/gpfs/bbp.cscs.ch/project/proj1/entities/bionames/SomatosensoryCxS1-v5.r0/bluerecipe_release_ChC_intervention_GSYNrescale/builderRecipeAllPathways.xml"
recipe_sdist = bbp.load_recipe_as_spatial_distribution(recipe_filename,annotation.raw, hierarchy, region_name)
sclass_sdist = tt.reduce_distribution_collection(recipe_sdist, 'sClass')

### run module

In [75]:
chosen_synapse_class = assign_synapse_class_from_spatial_dist(positions, sclass_sdist, voxel_dimensions)

### output

In [76]:
viewer.export_points(joinp(output_directory, 'sclass.pts'),positions, chosen_synapse_class, 'sClass')
HTML(iframe % "sclass.pts")

## Build.Composition.ME: METype for Soma

In [77]:
from brainbuilder.assignment_metype import assign_metype

### run module

In [78]:
chosen_me = assign_metype(positions, chosen_synapse_class, recipe_sdist, voxel_dimensions)

### output

#### mtypes

In [80]:
viewer.export_points(joinp(output_directory, 'mtype.pts'), positions, [me[0] for me in chosen_me], 'mtype')
HTML(iframe % 'mtype.pts')

#### etypes

In [81]:
viewer.export_points(joinp(output_directory, 'etype.pts'), positions, [me[1] for me in chosen_me], 'etype')
HTML(iframe % 'etype.pts')

## Build.Placement: Morphology assignment

In [82]:
from brainbuilder.assignment_morphology import assign_morphology

### input parameters

In [83]:
neurondb_filename = "/gpfs/bbp.cscs.ch/project/proj1/entities/morphologies/2012.07.23/h5/v4_24.09.14Final_NoBadClonesMEGatedAntiStuck0_NoL6MCsubs_ih_exp_Soma\
tosensoryCxS1-v5_r0_NeuronDB.dat"
neuron_sdist = bbp.load_neurondb_v4_as_spatial_distribution(neurondb_filename, annotation.raw,hierarchy, region_name,percentile=0.92)

### run module

In [84]:
chosen_morphology = assign_morphology(positions, chosen_me, neuron_sdist, voxel_dimensions)

In [87]:
viewer.export_points(joinp(output_directory, 'morph.pts'), positions, chosen_morphology, 'morphology')
HTML(iframe % 'morph.pts')